In [1]:
from bs4 import BeautifulSoup
# import pandas as pd
from datetime import datetime, timedelta
from itertools import combinations
import json
import requests
import math
from typing import Optional, List, Dict
import os

from pydantic import BaseModel
from typing import Mapping
import numpy as np
import polars as pl
import hvplot.polars
import pandas as pd
import altair as alt
import math

from shared.utils import formalize_data_with_str
from shared.random_rules import RandomRules

In [2]:
rr = RandomRules(
  TOTAL_NUMBERS = 50,
  CHOOSE = 6,
  MAX_PRIMES = 3,
  N_VALUES=["1", "2", "3"],
  # Rangos de valores de las bolillas
  MIN_B_VALUES = [1, 2, 3, 4, 5, 6],
  MAX_B_VALUES = [45, 46, 47, 48, 49, 50]
)

In [3]:
url = 'https://resultados.latinka.com.pe/i.do?m=historico&t=0&s=41'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

tables = soup.select('table')
rows = tables[1].find_all('tr')

In [4]:
lst = []
post_str_b1 = None
post_str_b2 = None
post_str_b3 = None
post_str_b4 = None
post_str_b5 = None
post_str_blast = None
for row in rows:
  cols = [data.text.strip() for data in row.find_all('td')]
  
  id, bolillas_int, str_combo = formalize_data_with_str(bolillas_str=cols[2])
  cols.append(id)
  cols.append(bolillas_int)
  cols.extend([b for b in bolillas_int])
  cols.extend(str_combo)
  cols.append(post_str_b1)
  cols.append(post_str_b2)
  cols.append(post_str_b3)
  cols.append(post_str_b4)
  cols.append(post_str_b5)
  cols.append(post_str_blast)

  is_rare, reason = rr.is_rare_combination(bolillas_int, id)
  cols.append(is_rare)
  cols.append(reason)

  cols.append(rr.jumps_map(bolillas_int))
  cols.append(rr.unijump(bolillas_int))
  
  lst.append(cols)

  post_str_b1 = str_combo[0]
  post_str_b2 = str_combo[1]
  post_str_b3 = str_combo[2]
  post_str_b4 = str_combo[3]
  post_str_b5 = str_combo[4]
  post_str_blast = str_combo[-1]

with open('./gn_tnk_v3.json', 'w') as file: 
  json.dump(lst, file)

dfRows = pl.DataFrame(
  lst,
  schema=[
    'fecha', 'sorteo', 'bolillas', 'yapa', 'adicionales', 'sorteo_extra',
    'id', 'combo', 'b1', 'b2', 'b3', 'b4', 'b5', 'b6',
    'str_b1', 'str_b2', 'str_b3', 'str_b4', 'str_b5', 'str_b6',
    'post_str_b1', 'post_str_b2', 'post_str_b3', 'post_str_b4', 'post_str_b5', 'post_str_blast',
    'is_rare_combination', 'reason',
    "jumps_map", "unijump"
  ],
  schema_overrides={
    'combo': pl.Array(pl.UInt8, 6),
    'b1': pl.UInt8,
    'b2': pl.UInt8,
    'b3': pl.UInt8,
    'b4': pl.UInt8,
    'b5': pl.UInt8,
    'b6': pl.UInt8,
    'reason': pl.UInt8
  },
  orient='row'
)

TOTAL_LINES = len(dfRows)
print(f'{TOTAL_LINES=}')
display(dfRows.head(10))
dfRows.write_parquet('dfrows_tnk.parquet')

TOTAL_LINES=2479


fecha,sorteo,bolillas,yapa,adicionales,sorteo_extra,id,combo,b1,b2,b3,b4,b5,b6,str_b1,str_b2,str_b3,str_b4,str_b5,str_b6,post_str_b1,post_str_b2,post_str_b3,post_str_b4,post_str_b5,post_str_blast,is_rare_combination,reason,jumps_map,unijump
str,str,str,str,str,str,str,"array[u8, 6]",u8,u8,u8,u8,u8,u8,str,str,str,str,str,str,str,str,str,str,str,str,bool,u8,str,i64
"""25/05/2025""","""1197""","""21 31 01 09 04 08""","""25""","""35 41 24""","""Promoción Sí o Sí""","""010408092131""","[1, 4, … 31]",1,4,8,9,21,31,"""01""","""04""","""08""","""09""","""21""","""31""",null,null,null,null,null,null,false,0,"""03_04_01_12_10""",-45
"""21/05/2025""","""1196""","""07 36 24 34 45 11""","""01""","""33 47""","""Promoción Sí o Sí""","""071124343645""","[7, 11, … 45]",7,11,24,34,36,45,"""07""","""11""","""24""","""34""","""36""","""45""","""01""","""04""","""08""","""09""","""21""","""31""",false,0,"""04_13_10_02_09""",13
"""18/05/2025""","""1195""","""11 13 42 09 33 15""","""46""","""12""","""Promoción Sí o Sí""","""091113153342""","[9, 11, … 42]",9,11,13,15,33,42,"""09""","""11""","""13""","""15""","""33""","""42""","""07""","""11""","""24""","""34""","""36""","""45""",true,8,"""02_02_02_18_09""",-57
"""14/05/2025""","""1194""","""38 07 12 13 04 09""","""42""","""46 28""","""Promoción Sí o Sí""","""040709121338""","[4, 7, … 38]",4,7,9,12,13,38,"""04""","""07""","""09""","""12""","""13""","""38""","""09""","""11""","""13""","""15""","""33""","""42""",false,0,"""03_02_03_01_25""",34
"""11/05/2025""","""1193""","""46 21 49 45 03 43""","""17""","""20 30 32 44""","""Promoción Sí o Sí""","""032143454649""","[3, 21, … 49]",3,21,43,45,46,49,"""03""","""21""","""43""","""45""","""46""","""49""","""04""","""07""","""09""","""12""","""13""","""38""",false,0,"""18_22_02_01_03""",-59
"""07/05/2025""","""1192""","""48 02 38 50 33 35""","""49""","""20 16""","""Promoción Sí o Sí""","""023335384850""","[2, 33, … 50]",2,33,35,38,48,50,"""02""","""33""","""35""","""38""","""48""","""50""","""03""","""21""","""43""","""45""","""46""","""49""",true,16,"""31_02_03_10_02""",3
"""04/05/2025""","""1191""","""09 35 19 15 37 10""","""46""","""25 02 38""","""Promoción Sí o Sí""","""091015193537""","[9, 10, … 37]",9,10,15,19,35,37,"""09""","""10""","""15""","""19""","""35""","""37""","""02""","""33""","""35""","""38""","""48""","""50""",true,16,"""01_05_04_16_02""",-57
"""30/04/2025""","""1190""","""11 36 33 06 02 16""","""37""","""14 07 10 01""","""Promoción Sí o Sí""","""020611163336""","[2, 6, … 36]",2,6,11,16,33,36,"""02""","""06""","""11""","""16""","""33""","""36""","""09""","""10""","""15""","""19""","""35""","""37""",true,8,"""04_05_05_17_03""",-51
"""27/04/2025""","""1189""","""46 33 21 08 07 41""","""39""","""36 09 37""","""Promoción Sí o Sí""","""070821334146""","[7, 8, … 46]",7,8,21,33,41,46,"""07""","""08""","""21""","""33""","""41""","""46""","""02""","""06""","""11""","""16""","""33""","""36""",false,0,"""01_13_12_08_05""",-6


In [5]:
dfRows.item(0, 'b2')

4

In [6]:
import holoviews as hv
def plot_post_col (col: str, post_col: str, test_value: str | None = None):
  if test_value == None:
    test_value = dfRows.item(0, col)
    print(f'Setting {test_value=}')

  df_viz = dfRows \
    .filter(
      pl.col(col) == test_value,
      pl.col(post_col).is_not_null()
    ) \
    .group_by(
      post_col
    ).agg(pl.count(post_col).alias('count')) \
    .sort("count", descending=True)
  
  
  df_10_last = dfRows \
    .filter(
      pl.col(col) == test_value,
      pl.col(post_col).is_not_null()
    ) \
    .head(10)
  
  # Agrega una columna que sea el porcentaje de la frecuencia relativa
  total = df_viz['count'].sum()
  df_viz = df_viz.with_columns(
    ( (pl.col('count') / total) * 100 ).alias('percent')
  )

  mean_percent = df_viz['percent'].mean()
  mean_percent_75 = mean_percent * 75 / 50
  mean_percent_25 = mean_percent * 25 / 50

  chart = df_viz.hvplot.bar(
    x=post_col,
    y='percent'
  )

  chart = chart * hv.HLine(mean_percent).opts(color='red')
  chart = chart * hv.HLine(mean_percent_75).opts(color='blue')
  chart = chart * hv.HLine(mean_percent_25).opts(color='blue')
  
  chart.opts(default_tools=["pan"])

  df_jumps_map = dfRows \
    .filter(
      pl.col(col) == test_value,
      pl.col(post_col).is_not_null()
    ) \
    .group_by(
      'jumps_map'
    ).agg(pl.count('jumps_map').alias('count')) \
    .sort("count", descending=True)
  
  df_unijump = dfRows \
    .filter(
      pl.col(col) == test_value,
      pl.col(post_col).is_not_null()
    ) \
    .group_by(
      'unijump'
    ).agg(pl.count('unijump').alias('count')) \
    .sort("count", descending=True)


  print(f'df_viz FOR {col} = {test_value}')
  display(df_viz)

  display(chart)
  

  print(f'df_10_last FOR {col} = {test_value}')
  last_results = list(set(df_10_last[post_col][0:3]))
  print(f'Last 3 results {last_results=}')
  display(df_10_last)
  
  print(f'df_jumps_map FOR {col} = {test_value}')
  display(df_jumps_map)
  # display(df_unijump)

  print(f'df_unijump FOR {col} = {test_value}')
  display(df_unijump)

  lst_post_str_b = df_viz.filter(pl.col('percent') >= mean_percent)[post_col].to_list()
  lst_jumps_map = df_jumps_map['jumps_map'].to_list() # Used to discard
  
  return lst_post_str_b, lst_jumps_map, last_results

In [7]:
possible_b1, exclude_jump_maps_b1, last_post_b1 = plot_post_col(col='str_b1', post_col='post_str_b1')

Setting test_value='01'
df_viz FOR str_b1 = 01


post_str_b1,count,percent
str,u32,f64
"""01""",58,17.210682
"""02""",42,12.462908
"""05""",30,8.902077
"""03""",28,8.308605
"""04""",22,6.52819
…,…,…
"""21""",2,0.593472
"""28""",1,0.296736
"""24""",1,0.296736


:Overlay
   .Bars.I    :Bars   [post_str_b1]   (percent)
   .HLine.I   :HLine   [x,y]
   .HLine.II  :HLine   [x,y]
   .HLine.III :HLine   [x,y]

df_10_last FOR str_b1 = 01
Last 3 results last_results=['07', '25', '22']


fecha,sorteo,bolillas,yapa,adicionales,sorteo_extra,id,combo,b1,b2,b3,b4,b5,b6,str_b1,str_b2,str_b3,str_b4,str_b5,str_b6,post_str_b1,post_str_b2,post_str_b3,post_str_b4,post_str_b5,post_str_blast,is_rare_combination,reason,jumps_map,unijump
str,str,str,str,str,str,str,"array[u8, 6]",u8,u8,u8,u8,u8,u8,str,str,str,str,str,str,str,str,str,str,str,str,bool,u8,str,i64
"""09/04/2025""","""1184""","""35 24 49 01 12 39""","""21""","""13 07 26""","""Promoción Sí o Sí""","""011224353949""","[1, 12, … 49]",1,12,24,35,39,49,"""01""","""12""","""24""","""35""","""39""","""49""","""22""","""36""","""39""","""44""","""46""","""48""",false,0,"""11_12_11_04_10""",23
"""26/03/2025""","""1180""","""43 01 05 37 35 13""","""39""","""38 07 45 16""","""Promoción Sí o Sí""","""010513353743""","[1, 5, … 43]",1,5,13,35,37,43,"""01""","""05""","""13""","""35""","""37""","""43""","""25""","""28""","""29""","""31""","""46""","""47""",true,19,"""04_08_22_02_06""",102
"""09/03/2025""","""1175""","""01 42 44 20 38 23""","""14""","""35 36 34 45 41""","""Promoción Sí o Sí""","""012023384244""","[1, 20, … 44]",1,20,23,38,42,44,"""01""","""20""","""23""","""38""","""42""","""44""","""07""","""16""","""17""","""32""","""34""","""44""",false,0,"""19_03_15_04_02""",83
"""05/03/2025""","""1174""","""01 32 03 12 28 47""","""20""","""49 06 23 24""","""Promoción Sí o Sí""","""010312283247""","[1, 3, … 47]",1,3,12,28,32,47,"""01""","""03""","""12""","""28""","""32""","""47""","""01""","""20""","""23""","""38""","""42""","""44""",false,0,"""02_09_16_04_15""",61
"""19/01/2025""","""1161""","""23 44 45 05 02 01""","""16""","""24 19""","""Promoción Sí o Sí""","""010205234445""","[1, 2, … 45]",1,2,5,23,44,45,"""01""","""02""","""05""","""23""","""44""","""45""","""06""","""10""","""21""","""31""","""32""","""49""",false,0,"""01_03_18_21_01""",14
"""22/12/2024""","""1153""","""17 02 49 40 24 01""","""45""","""14 22 05""","""Promoción Sí o Sí""","""010217244049""","[1, 2, … 49]",1,2,17,24,40,49,"""01""","""02""","""17""","""24""","""40""","""49""","""02""","""05""","""14""","""15""","""24""","""39""",true,16,"""01_15_07_16_09""",-72
"""23/10/2024""","""1136""","""28 09 16 01 46 40""","""50""","""37 41 27""","""Promoción Sí o Sí""","""010916284046""","[1, 9, … 46]",1,9,16,28,40,46,"""01""","""09""","""16""","""28""","""40""","""46""","""06""","""16""","""24""","""27""","""32""","""47""",true,24,"""08_07_12_12_06""",10
"""15/09/2024""","""1125""","""36 43 29 01 03 38""","""25""","""19 18 35 02""","""Promoción Sí o Sí""","""010329363843""","[1, 3, … 43]",1,3,29,36,38,43,"""01""","""03""","""29""","""36""","""38""","""43""","""10""","""21""","""26""","""32""","""34""","""37""",false,0,"""02_26_07_02_05""",-63
"""25/08/2024""","""1119""","""49 01 42 14 27 36""","""05""","""21 45""","""Promoción Sí o Sí""","""011427364249""","[1, 14, … 49]",1,14,27,36,42,49,"""01""","""14""","""27""","""36""","""42""","""49""","""03""","""06""","""11""","""37""","""46""","""47""",true,8,"""13_13_09_06_07""",-2


df_jumps_map FOR str_b1 = 01


jumps_map,count
str,u32
"""07_05_06_02_14""",1
"""02_03_18_01_14""",1
"""07_06_03_03_07""",1
"""03_12_03_05_11""",1
"""01_09_03_02_29""",1
…,…
"""01_03_18_21_01""",1
"""15_17_01_08_02""",1
"""10_11_01_04_06""",1


df_unijump FOR str_b1 = 01


unijump,count
i64,u32
-20,10
1,7
11,6
-16,6
-26,6
…,…
-8,1
88,1
-28,1


In [8]:
possible_b2, exclude_jump_maps_b2, last_post_b2 = plot_post_col(col='str_b2', post_col='post_str_b2')

Setting test_value='04'
df_viz FOR str_b2 = 04


post_str_b2,count,percent
str,u32,f64
"""15""",9,9.090909
"""18""",8,8.080808
"""16""",7,7.070707
"""13""",6,6.060606
"""12""",6,6.060606
…,…,…
"""11""",2,2.020202
"""02""",2,2.020202
"""28""",1,1.010101


:Overlay
   .Bars.I    :Bars   [post_str_b2]   (percent)
   .HLine.I   :HLine   [x,y]
   .HLine.II  :HLine   [x,y]
   .HLine.III :HLine   [x,y]

df_10_last FOR str_b2 = 04
Last 3 results last_results=['15', '20']


fecha,sorteo,bolillas,yapa,adicionales,sorteo_extra,id,combo,b1,b2,b3,b4,b5,b6,str_b1,str_b2,str_b3,str_b4,str_b5,str_b6,post_str_b1,post_str_b2,post_str_b3,post_str_b4,post_str_b5,post_str_blast,is_rare_combination,reason,jumps_map,unijump
str,str,str,str,str,str,str,"array[u8, 6]",u8,u8,u8,u8,u8,u8,str,str,str,str,str,str,str,str,str,str,str,str,bool,u8,str,i64
"""03/11/2024""","""1139""","""04 24 37 36 14 02""","""03""","""50 12 42 10""","""Promoción Sí o Sí""","""020414243637""","[2, 4, … 37]",2,4,14,24,36,37,"""02""","""04""","""14""","""24""","""36""","""37""","""02""","""15""","""17""","""40""","""46""","""48""",true,8,"""02_10_10_12_01""",-25
"""08/05/2024""","""1088""","""04 45 01 47 27 20""","""46""","""44""","""Promoción Sí o Sí""","""010420274547""","[1, 4, … 47]",1,4,20,27,45,47,"""01""","""04""","""20""","""27""","""45""","""47""","""10""","""20""","""25""","""37""","""43""","""46""",true,16,"""03_16_07_18_02""",-89
"""24/01/2024""","""1058""","""04 01 46 24 38 39""","""25""","""21""","""Promoción Sí o Sí""","""010424383946""","[1, 4, … 46]",1,4,24,38,39,46,"""01""","""04""","""24""","""38""","""39""","""46""","""03""","""15""","""19""","""21""","""24""","""25""",false,0,"""03_20_14_01_07""",10
"""07/01/2024""","""1053""","""18 27 04 33 07 01""","""31""","""23 02 37""","""Promoción Sí o Sí""","""010407182733""","[1, 4, … 33]",1,4,7,18,27,33,"""01""","""04""","""07""","""18""","""27""","""33""","""01""","""10""","""11""","""29""","""30""","""36""",true,8,"""03_03_11_09_06""",27
"""07/05/2023""","""0983""","""05 22 04 01 09 23""","""34""","""35 44""","""Promoción Sí o Sí""","""010405092223""","[1, 4, … 23]",1,4,5,9,22,23,"""01""","""04""","""05""","""09""","""22""","""23""","""09""","""15""","""16""","""20""","""22""","""34""",false,0,"""03_01_04_13_01""",-28
"""29/03/2023""","""0972""","""14 04 08 23 03 18""","""42""","""45 26""","""Promoción Sí o Sí""","""030408141823""","[3, 4, … 23]",3,4,8,14,18,23,"""03""","""04""","""08""","""14""","""18""","""23""","""02""","""14""","""24""","""28""","""31""","""34""",false,0,"""01_04_06_04_05""",10
"""22/02/2023""","""0962""","""24 29 19 04 03 43""","""11""","""08 20""","""Promoción Sí o Sí""","""030419242943""","[3, 4, … 43]",3,4,19,24,29,43,"""03""","""04""","""19""","""24""","""29""","""43""","""02""","""05""","""16""","""28""","""43""","""46""",true,9,"""01_15_05_05_14""",-35
"""08/01/2023""","""0949""","""04 44 22 10 03 42""","""38""","""33 46 17""","""Promoción Sí o Sí""","""030410224244""","[3, 4, … 44]",3,4,10,22,42,44,"""03""","""04""","""10""","""22""","""42""","""44""","""15""","""25""","""28""","""34""","""38""","""43""",false,0,"""01_06_12_20_02""",-29
"""04/01/2023""","""0948""","""45 04 38 01 14 17""","""07""","""34 42""","""Promoción Sí o Sí""","""010414173845""","[1, 4, … 45]",1,4,14,17,38,45,"""01""","""04""","""14""","""17""","""38""","""45""","""03""","""04""","""10""","""22""","""42""","""44""",true,16,"""03_10_03_21_07""",-96


df_jumps_map FOR str_b2 = 04


jumps_map,count
str,u32
"""01_01_08_14_10""",1
"""01_02_04_06_13""",1
"""02_17_09_06_06""",1
"""01_05_21_10_04""",1
"""03_02_09_01_02""",1
…,…
"""01_03_09_27_02""",1
"""02_04_17_04_06""",1
"""01_21_15_01_01""",1


df_unijump FOR str_b2 = 04


unijump,count
i64,u32
79,3
-25,3
1,3
-21,3
6,3
…,…
50,1
112,1
20,1


In [9]:
possible_b3, exclude_jump_maps_b3, last_post_b3 = plot_post_col(col='str_b3', post_col='post_str_b3')

Setting test_value='08'
df_viz FOR str_b3 = 08


post_str_b3,count,percent
str,u32,f64
"""14""",10,17.857143
"""21""",6,10.714286
"""15""",4,7.142857
"""28""",4,7.142857
"""13""",3,5.357143
…,…,…
"""32""",1,1.785714
"""22""",1,1.785714
"""12""",1,1.785714


:Overlay
   .Bars.I    :Bars   [post_str_b3]   (percent)
   .HLine.I   :HLine   [x,y]
   .HLine.II  :HLine   [x,y]
   .HLine.III :HLine   [x,y]

df_10_last FOR str_b3 = 08
Last 3 results last_results=['23', '28', '14']


fecha,sorteo,bolillas,yapa,adicionales,sorteo_extra,id,combo,b1,b2,b3,b4,b5,b6,str_b1,str_b2,str_b3,str_b4,str_b5,str_b6,post_str_b1,post_str_b2,post_str_b3,post_str_b4,post_str_b5,post_str_blast,is_rare_combination,reason,jumps_map,unijump
str,str,str,str,str,str,str,"array[u8, 6]",u8,u8,u8,u8,u8,u8,str,str,str,str,str,str,str,str,str,str,str,str,bool,u8,str,i64
"""15/12/2024""","""1151""","""48 26 45 08 06 04""","""44""","""15 29""","""Promoción Sí o Sí""","""040608264548""","[4, 6, … 48]",4,6,8,26,45,48,"""04""","""06""","""08""","""26""","""45""","""48""","""19""","""22""","""28""","""33""","""35""","""47""",true,24,"""02_02_18_19_03""",29
"""06/10/2024""","""1131""","""14 09 02 25 08 06""","""44""","""05 19 15""","""Promoción Sí o Sí""","""020608091425""","[2, 6, … 25]",2,6,8,9,14,25,"""02""","""06""","""08""","""09""","""14""","""25""","""15""","""20""","""23""","""27""","""37""","""48""",true,16,"""04_02_01_05_11""",-7
"""03/12/2023""","""1043""","""22 08 03 01 27 19""","""34""","""16""","""Promoción Sí o Sí""","""010308192227""","[1, 3, … 27]",1,3,8,19,22,27,"""01""","""03""","""08""","""19""","""22""","""27""","""02""","""12""","""14""","""15""","""18""","""27""",false,0,"""02_05_11_03_05""",41
"""17/09/2023""","""1021""","""36 05 47 46 07 08""","""14""","""02 01 18""","""Promoción Sí o Sí""","""050708364647""","[5, 7, … 47]",5,7,8,36,46,47,"""05""","""07""","""08""","""36""","""46""","""47""","""19""","""24""","""27""","""34""","""42""","""47""",true,16,"""02_01_28_10_01""",127
"""29/03/2023""","""0972""","""14 04 08 23 03 18""","""42""","""45 26""","""Promoción Sí o Sí""","""030408141823""","[3, 4, … 23]",3,4,8,14,18,23,"""03""","""04""","""08""","""14""","""18""","""23""","""02""","""14""","""24""","""28""","""31""","""34""",false,0,"""01_04_06_04_05""",10
"""03/04/2022""","""0869""","""04 35 34 08 05 36""","""30""","""19 14""","""Promoción Sí o Sí""","""040508343536""","[4, 5, … 36]",4,5,8,34,35,36,"""04""","""05""","""08""","""34""","""35""","""36""","""05""","""10""","""13""","""15""","""18""","""31""",true,24,"""01_03_26_01_01""",142
"""16/03/2022""","""0864""","""39 06 02 08 24 26""","""23""","""11 33""","""Promoción Sí o Sí""","""020608242639""","[2, 6, … 39]",2,6,8,24,26,39,"""02""","""06""","""08""","""24""","""26""","""39""","""04""","""19""","""21""","""23""","""33""","""46""",true,16,"""04_02_16_02_13""",97
"""27/10/2021""","""0824""","""01 38 09 06 08 42""","""21""","""""","""""","""010608093842""","[1, 6, … 42]",1,6,8,9,38,42,"""01""","""06""","""08""","""09""","""38""","""42""","""28""","""29""","""31""","""33""","""35""","""39""",false,0,"""05_02_01_29_04""",-109
"""14/07/2021""","""0794""","""11 42 09 01 08 03""","""19""","""""","""""","""010308091142""","[1, 3, … 42]",1,3,8,9,11,42,"""01""","""03""","""08""","""09""","""11""","""42""","""02""","""04""","""16""","""34""","""36""","""39""",false,0,"""02_05_01_02_31""",11


df_jumps_map FOR str_b3 = 08


jumps_map,count
str,u32
"""05_02_01_29_04""",1
"""02_01_28_10_01""",1
"""03_04_05_02_27""",1
"""02_02_01_06_07""",1
"""04_02_16_02_13""",1
…,…
"""05_01_02_16_09""",1
"""03_01_16_12_06""",1
"""03_03_06_11_12""",1


df_unijump FOR str_b3 = 08


unijump,count
i64,u32
80,2
127,2
41,1
39,1
-66,1
…,…
142,1
53,1
28,1


In [10]:
possible_b4, exclude_jump_maps_b4, last_post_b4 = plot_post_col(col='str_b4', post_col='post_str_b4')

Setting test_value='09'
df_viz FOR str_b4 = 09


post_str_b4,count,percent
str,u32,f64
"""33""",3,20.0
"""22""",2,13.333333
"""30""",2,13.333333
"""31""",1,6.666667
"""16""",1,6.666667
…,…,…
"""14""",1,6.666667
"""27""",1,6.666667
"""17""",1,6.666667


:Overlay
   .Bars.I    :Bars   [post_str_b4]   (percent)
   .HLine.I   :HLine   [x,y]
   .HLine.II  :HLine   [x,y]
   .HLine.III :HLine   [x,y]

df_10_last FOR str_b4 = 09
Last 3 results last_results=['20', '30', '27']


fecha,sorteo,bolillas,yapa,adicionales,sorteo_extra,id,combo,b1,b2,b3,b4,b5,b6,str_b1,str_b2,str_b3,str_b4,str_b5,str_b6,post_str_b1,post_str_b2,post_str_b3,post_str_b4,post_str_b5,post_str_blast,is_rare_combination,reason,jumps_map,unijump
str,str,str,str,str,str,str,"array[u8, 6]",u8,u8,u8,u8,u8,u8,str,str,str,str,str,str,str,str,str,str,str,str,bool,u8,str,i64
"""06/10/2024""","""1131""","""14 09 02 25 08 06""","""44""","""05 19 15""","""Promoción Sí o Sí""","""020608091425""","[2, 6, … 25]",2,6,8,9,14,25,"""02""","""06""","""08""","""09""","""14""","""25""","""15""","""20""","""23""","""27""","""37""","""48""",true,16,"""04_02_01_05_11""",-7
"""07/05/2023""","""0983""","""05 22 04 01 09 23""","""34""","""35 44""","""Promoción Sí o Sí""","""010405092223""","[1, 4, … 23]",1,4,5,9,22,23,"""01""","""04""","""05""","""09""","""22""","""23""","""09""","""15""","""16""","""20""","""22""","""34""",false,0,"""03_01_04_13_01""",-28
"""26/10/2022""","""0928""","""04 09 07 12 02 27""","""45""","""41 03""","""Promoción Sí o Sí""","""020407091227""","[2, 4, … 27]",2,4,7,9,12,27,"""02""","""04""","""07""","""09""","""12""","""27""","""10""","""14""","""22""","""30""","""35""","""44""",true,16,"""02_03_02_03_15""",5
"""27/04/2022""","""0876""","""09 21 42 01 05 03""","""19""","""18 22""","""Promoción Sí o Sí""","""010305092142""","[1, 3, … 42]",1,3,5,9,21,42,"""01""","""03""","""05""","""09""","""21""","""42""","""16""","""24""","""26""","""33""","""39""","""43""",true,8,"""02_02_04_12_21""",-9
"""27/10/2021""","""0824""","""01 38 09 06 08 42""","""21""","""""","""""","""010608093842""","[1, 6, … 42]",1,6,8,9,38,42,"""01""","""06""","""08""","""09""","""38""","""42""","""28""","""29""","""31""","""33""","""35""","""39""",false,0,"""05_02_01_29_04""",-109
"""14/07/2021""","""0794""","""11 42 09 01 08 03""","""19""","""""","""""","""010308091142""","[1, 3, … 42]",1,3,8,9,11,42,"""01""","""03""","""08""","""09""","""11""","""42""","""02""","""04""","""16""","""34""","""36""","""39""",false,0,"""02_05_01_02_31""",11
"""22/09/2019""","""0638""","""03 09 19 08 17 02""","""27""","""""","""""","""020308091719""","[2, 3, … 19]",2,3,8,9,17,19,"""02""","""03""","""08""","""09""","""17""","""19""","""04""","""16""","""28""","""30""","""34""","""40""",true,1,"""01_05_01_08_02""",-43
"""30/03/2016""","""0275""","""01 07 08 09 26 34""","""37""","""""","""""","""010708092634""","[1, 7, … 34]",1,7,8,9,26,34,"""01""","""07""","""08""","""09""","""26""","""34""","""04""","""10""","""14""","""25""","""30""","""40""",true,8,"""06_01_01_17_08""",-52
"""14/10/2009""","""1067""","""05 07 08 09 21 24""","""26""","""28 31 36 43""","""Promoción Sí o Sí""","""050708092124""","[5, 7, … 24]",5,7,8,9,21,24,"""05""","""07""","""08""","""09""","""21""","""24""","""05""","""13""","""14""","""22""","""40""","""43""",true,24,"""02_01_01_12_03""",-41


df_jumps_map FOR str_b4 = 09


jumps_map,count
str,u32
"""01_05_01_08_02""",1
"""01_02_01_05_20""",1
"""02_01_01_12_03""",1
"""02_02_04_12_21""",1
"""05_02_01_29_04""",1
…,…
"""02_05_01_08_20""",1
"""06_01_01_17_08""",1
"""02_03_01_05_01""",1


df_unijump FOR str_b4 = 09


unijump,count
i64,u32
-112,1
-52,1
-24,1
-1,1
-9,1
…,…
-17,1
-109,1
-41,1


In [11]:
possible_b5, exclude_jump_maps_b5, last_post_b5 = plot_post_col(col='str_b5', post_col='post_str_b5')

Setting test_value='21'
df_viz FOR str_b5 = 21


post_str_b5,count,percent
str,u32,f64
"""34""",5,11.627907
"""33""",5,11.627907
"""32""",4,9.302326
"""39""",3,6.976744
"""37""",3,6.976744
…,…,…
"""42""",1,2.325581
"""44""",1,2.325581
"""26""",1,2.325581


:Overlay
   .Bars.I    :Bars   [post_str_b5]   (percent)
   .HLine.I   :HLine   [x,y]
   .HLine.II  :HLine   [x,y]
   .HLine.III :HLine   [x,y]

df_10_last FOR str_b5 = 21
Last 3 results last_results=['39', '33', '43']


fecha,sorteo,bolillas,yapa,adicionales,sorteo_extra,id,combo,b1,b2,b3,b4,b5,b6,str_b1,str_b2,str_b3,str_b4,str_b5,str_b6,post_str_b1,post_str_b2,post_str_b3,post_str_b4,post_str_b5,post_str_blast,is_rare_combination,reason,jumps_map,unijump
str,str,str,str,str,str,str,"array[u8, 6]",u8,u8,u8,u8,u8,u8,str,str,str,str,str,str,str,str,str,str,str,str,bool,u8,str,i64
"""09/04/2023""","""0975""","""02 19 21 25 18 08""","""35""","""28 34""","""Promoción Sí o Sí""","""020818192125""","[2, 8, … 25]",2,8,18,19,21,25,"""02""","""08""","""18""","""19""","""21""","""25""","""06""","""08""","""22""","""25""","""33""","""38""",true,16,"""06_10_01_02_04""",-32
"""22/05/2022""","""0883""","""11 10 12 21 01 26""","""36""","""33 38""","""Promoción Sí o Sí""","""011011122126""","[1, 10, … 26]",1,10,11,12,21,26,"""01""","""10""","""11""","""12""","""21""","""26""","""12""","""28""","""39""","""42""","""43""","""45""",true,24,"""09_01_01_09_05""",-20
"""27/04/2022""","""0876""","""09 21 42 01 05 03""","""19""","""18 22""","""Promoción Sí o Sí""","""010305092142""","[1, 3, … 42]",1,3,5,9,21,42,"""01""","""03""","""05""","""09""","""21""","""42""","""16""","""24""","""26""","""33""","""39""","""43""",true,8,"""02_02_04_12_21""",-9
"""23/02/2022""","""0858""","""13 35 19 01 21 08""","""10""","""45 44 16""","""Promoción Sí o Sí""","""010813192135""","[1, 8, … 35]",1,8,13,19,21,35,"""01""","""08""","""13""","""19""","""21""","""35""","""10""","""11""","""21""","""23""","""30""","""45""",false,0,"""07_05_06_02_14""",29
"""23/01/2022""","""0849""","""21 19 14 01 08 26""","""22""","""""","""""","""010814192126""","[1, 8, … 26]",1,8,14,19,21,26,"""01""","""08""","""14""","""19""","""21""","""26""","""11""","""26""","""30""","""33""","""40""","""43""",true,16,"""07_06_05_02_05""",10
"""21/03/2018""","""0481""","""21 02 19 20 17 33""","""05""","""""","""""","""021719202133""","[2, 17, … 33]",2,17,19,20,21,33,"""02""","""17""","""19""","""20""","""21""","""33""","""02""","""06""","""08""","""20""","""41""","""43""",true,8,"""15_02_01_01_12""",21
"""01/11/2017""","""0441""","""16 01 21 11 15 25""","""13""","""""","""""","""011115162125""","[1, 11, … 25]",1,11,15,16,21,25,"""01""","""11""","""15""","""16""","""21""","""25""","""05""","""13""","""19""","""22""","""34""","""44""",true,16,"""10_04_01_05_04""",-16
"""18/09/2016""","""0324""","""05 06 11 13 21 42""","""40""","""""","""""","""050611132142""","[5, 6, … 42]",5,6,11,13,21,42,"""05""","""06""","""11""","""13""","""21""","""42""","""11""","""17""","""20""","""35""","""36""","""37""",false,0,"""01_05_02_08_21""",-18
"""17/06/2015""","""0193""","""01 05 10 14 21 22""","""32""","""""","""""","""010510142122""","[1, 5, … 22]",1,5,10,14,21,22,"""01""","""05""","""10""","""14""","""21""","""22""","""09""","""11""","""12""","""15""","""32""","""33""",true,16,"""04_05_04_07_01""",-19


df_jumps_map FOR str_b5 = 21


jumps_map,count
str,u32
"""04_04_02_06_07""",1
"""02_05_01_05_04""",1
"""10_04_02_03_16""",1
"""02_05_02_05_18""",1
"""02_03_01_01_24""",1
…,…
"""01_09_04_06_03""",1
"""09_01_01_09_05""",1
"""05_04_04_04_09""",1


df_unijump FOR str_b5 = 21


unijump,count
i64,u32
10,3
2,2
-16,2
-32,2
11,2
…,…
-28,1
-20,1
-33,1


In [12]:
possible_blast, exclude_jump_maps_blast, last_post_blast = plot_post_col(col='str_b6', post_col='post_str_blast')

Setting test_value='31'
df_viz FOR str_b6 = 31


post_str_blast,count,percent
str,u32,f64
"""45""",7,12.280702
"""41""",6,10.526316
"""34""",5,8.77193
"""43""",4,7.017544
"""39""",4,7.017544
…,…,…
"""26""",1,1.754386
"""24""",1,1.754386
"""27""",1,1.754386


:Overlay
   .Bars.I    :Bars   [post_str_blast]   (percent)
   .HLine.I   :HLine   [x,y]
   .HLine.II  :HLine   [x,y]
   .HLine.III :HLine   [x,y]

df_10_last FOR str_b6 = 31
Last 3 results last_results=['47', '48']


fecha,sorteo,bolillas,yapa,adicionales,sorteo_extra,id,combo,b1,b2,b3,b4,b5,b6,str_b1,str_b2,str_b3,str_b4,str_b5,str_b6,post_str_b1,post_str_b2,post_str_b3,post_str_b4,post_str_b5,post_str_blast,is_rare_combination,reason,jumps_map,unijump
str,str,str,str,str,str,str,"array[u8, 6]",u8,u8,u8,u8,u8,u8,str,str,str,str,str,str,str,str,str,str,str,str,bool,u8,str,i64
"""13/10/2024""","""1133""","""31 17 28 09 21 05""","""08""","""22 27""","""Promoción Sí o Sí""","""050917212831""","[5, 9, … 31]",5,9,17,21,28,31,"""05""","""09""","""17""","""21""","""28""","""31""","""08""","""10""","""12""","""29""","""31""","""47""",false,0,"""04_08_04_07_03""",-29
"""20/03/2024""","""1074""","""22 08 31 11 10 21""","""06""","""16 43""","""Promoción Sí o Sí""","""081011212231""","[8, 10, … 31]",8,10,11,21,22,31,"""08""","""10""","""11""","""21""","""22""","""31""","""09""","""21""","""25""","""28""","""32""","""48""",false,0,"""02_01_10_01_09""",63
"""19/11/2023""","""1039""","""08 23 21 31 19 20""","""16""","""28 33""","""Promoción Sí o Sí""","""081920212331""","[8, 19, … 31]",8,19,20,21,23,31,"""08""","""19""","""20""","""21""","""23""","""31""","""09""","""22""","""23""","""29""","""43""","""48""",true,8,"""11_01_01_02_08""",13
"""27/07/2022""","""0902""","""11 29 06 02 31 27""","""33""","""25 37""","""Promoción Sí o Sí""","""020611272931""","[2, 6, … 31]",2,6,11,27,29,31,"""02""","""06""","""11""","""27""","""29""","""31""","""12""","""19""","""20""","""23""","""25""","""41""",true,9,"""04_05_16_02_02""",74
"""24/07/2022""","""0901""","""07 23 04 12 03 31""","""22""","""41""","""Promoción Sí o Sí""","""030407122331""","[3, 4, … 31]",3,4,7,12,23,31,"""03""","""04""","""07""","""12""","""23""","""31""","""02""","""06""","""11""","""27""","""29""","""31""",true,1,"""01_03_05_11_08""",-17
"""06/04/2022""","""0870""","""31 10 13 05 15 18""","""43""","""38 28""","""Promoción Sí o Sí""","""051013151831""","[5, 10, … 31]",5,10,13,15,18,31,"""05""","""10""","""13""","""15""","""18""","""31""","""10""","""19""","""25""","""32""","""35""","""42""",true,16,"""05_03_02_03_13""",6
"""21/04/2021""","""0770""","""16 12 31 24 20 28""","""40""","""""","""""","""121620242831""","[12, 16, … 31]",12,16,20,24,28,31,"""12""","""16""","""20""","""24""","""28""","""31""","""23""","""26""","""30""","""33""","""39""","""41""",true,8,"""04_04_04_04_03""",-1
"""09/12/2020""","""0732""","""07 10 30 31 27 15""","""05""","""""","""""","""071015273031""","[7, 10, … 31]",7,10,15,27,30,31,"""07""","""10""","""15""","""27""","""30""","""31""","""02""","""13""","""23""","""24""","""25""","""34""",false,0,"""03_05_12_03_01""",44
"""02/12/2018""","""0554""","""18 23 01 31 03 27""","""33""","""""","""""","""010318232731""","[1, 3, … 31]",1,3,18,23,27,31,"""01""","""03""","""18""","""23""","""27""","""31""","""03""","""09""","""14""","""24""","""29""","""45""",true,8,"""02_15_05_04_04""",-40


df_jumps_map FOR str_b6 = 31


jumps_map,count
str,u32
"""01_02_08_01_06""",1
"""08_11_01_02_04""",1
"""02_01_10_01_09""",1
"""15_07_02_03_02""",1
"""04_01_07_10_02""",1
…,…
"""04_06_04_02_13""",1
"""04_07_04_06_01""",1
"""04_03_14_05_01""",1


df_unijump FOR str_b6 = 31


unijump,count
i64,u32
-30,3
-9,2
-38,2
4,2
-5,2
…,…
-40,1
9,1
-41,1


In [13]:
b_count = rr.CHOOSE
min_b = rr.MIN_B_VALUES[0]
max_b = rr.MAX_B_VALUES[-1]


combos = combinations(range(min_b, max_b + 1), b_count)
# print("generating combos: ", len(list(combos)))

In [14]:
parquet_file = './parquet_tnk_v3.parquet'

# check if parquet_file exists
if not os.path.exists(parquet_file):
  print(f'{parquet_file=} does not exists. creating')
  lst = []
  post_str_b1 = None
  post_str_blast = None
  for row in combos:
    cols = []
    str_row = " ".join([str(r).zfill(2) for r in row])
    id, bolillas_int, str_combo = formalize_data_with_str(bolillas_str=str_row)
    cols.append(id)
    cols.append(bolillas_int)
    cols.extend([b for b in bolillas_int])
    cols.extend(str_combo)
    cols.append(post_str_b1)
    cols.append(post_str_blast)

    is_rare, reason = rr.is_rare_combination(bolillas_int, id)
    cols.append(is_rare)
    cols.append(reason)

    cols.append(rr.jumps_map(bolillas_int))
    cols.append(rr.unijump(bolillas_int))
    
    lst.append(cols)

    post_str_b1 = str_combo[0]
    post_str_blast = str_combo[-1]

  df_Orows = pl.DataFrame(
    lst,
    schema=[
      # 'fecha', 'sorteo', 'bolillas',
      'id', 'combo', 'b1', 'b2', 'b3', 'b4', 'b5', 'b6',
      'str_b1', 'str_b2', 'str_b3', 'str_b4', 'str_b5', 'str_b6',
      'post_str_b1', 'post_str_blast',
      'is_rare_combination', 'reason',
      "jumps_map", "unijump"
    ],
    schema_overrides={
      'combo': pl.Array(pl.UInt8, 6),
      'b1': pl.UInt8,
      'b2': pl.UInt8,
      'b3': pl.UInt8,
      'b4': pl.UInt8,
      'b5': pl.UInt8,
      'b6': pl.UInt8,
      'reason': pl.UInt8
    },
    orient='row'
  )

  df_Orows.write_parquet(parquet_file)
  TOTAL_LINES = len(df_Orows)
  print(f'{TOTAL_LINES=} loaded to {parquet_file=}')

  display(df_Orows.head(10))
else:
  df_Orows = pl.read_parquet(parquet_file)

In [28]:
# possible_b1, exclude_jump_maps_b1
df_possible = df_Orows.filter(
  pl.col("str_b1").is_in(["10", "11", "12", "13", "14", "15", "16"]),
  # pl.col("str_b1").is_in(possible_b1),
  # pl.col("str_b2").is_in(possible_b2),
  # pl.col("str_b3").is_in(possible_b3),
  # pl.col("str_b4").is_in(possible_b4),
  # pl.col("str_b5").is_in(possible_b5),
  # pl.col("str_b6").is_in(possible_blast),
  
  # ~pl.col("str_b1").is_in(last_post_b1),
  ~pl.col("str_b2").is_in(last_post_b2 + ["40", "41", "42", "43", "44", "45", "46", "47", "48", "49"]),
  ~pl.col("str_b3").is_in(last_post_b3 + ["40", "41", "42", "43", "44", "45", "46", "47", "48", "49"]),
  ~pl.col("str_b4").is_in(last_post_b4 + ["10", "11", "12", "13", "14", "15", "16", "17", "18", "19", "20", "21", "22", "23", "24", "25"]),
  ~pl.col("str_b5").is_in(last_post_b5 + ["27"]),
  ~pl.col("str_b6").is_in(last_post_blast + ["20", "21", "22", "23", "24", "25", "26", "27", "28", "29", "30", "31", "32", "33", "34"]),

  ~pl.col("jumps_map").is_in(exclude_jump_maps_b1 + exclude_jump_maps_b2 + exclude_jump_maps_b3 + exclude_jump_maps_b4 + exclude_jump_maps_b5 + exclude_jump_maps_blast),
  # ~pl.col("jumps_map").is_in(exclude_jump_maps_b1 + exclude_jump_maps_b2 + exclude_jump_maps_blast),
  pl.col("is_rare_combination") == False
)


display(df_possible)

id,combo,b1,b2,b3,b4,b5,b6,str_b1,str_b2,str_b3,str_b4,str_b5,str_b6,post_str_b1,post_str_blast,is_rare_combination,reason,jumps_map,unijump
str,"array[u8, 6]",u8,u8,u8,u8,u8,u8,str,str,str,str,str,str,str,str,bool,u8,str,i64
"""101113262835""","[10, 11, … 35]",10,11,13,26,28,35,"""10""","""11""","""13""","""26""","""28""","""35""","""10""","""34""",false,0,"""01_02_13_02_07""",70
"""101113262836""","[10, 11, … 36]",10,11,13,26,28,36,"""10""","""11""","""13""","""26""","""28""","""36""","""10""","""35""",false,0,"""01_02_13_02_08""",71
"""101113262837""","[10, 11, … 37]",10,11,13,26,28,37,"""10""","""11""","""13""","""26""","""28""","""37""","""10""","""36""",false,0,"""01_02_13_02_09""",72
"""101113262838""","[10, 11, … 38]",10,11,13,26,28,38,"""10""","""11""","""13""","""26""","""28""","""38""","""10""","""37""",false,0,"""01_02_13_02_10""",73
"""101113262839""","[10, 11, … 39]",10,11,13,26,28,39,"""10""","""11""","""13""","""26""","""28""","""39""","""10""","""38""",false,0,"""01_02_13_02_11""",74
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""163839424749""","[16, 38, … 49]",16,38,39,42,47,49,"""16""","""38""","""39""","""42""","""47""","""49""","""16""","""48""",false,0,"""22_01_03_05_02""",18
"""163839424750""","[16, 38, … 50]",16,38,39,42,47,50,"""16""","""38""","""39""","""42""","""47""","""50""","""16""","""49""",false,0,"""22_01_03_05_03""",19
"""163839424849""","[16, 38, … 49]",16,38,39,42,48,49,"""16""","""38""","""39""","""42""","""48""","""49""","""16""","""50""",false,0,"""22_01_03_06_01""",13


In [29]:
df_possible.sample(n=1)


id,combo,b1,b2,b3,b4,b5,b6,str_b1,str_b2,str_b3,str_b4,str_b5,str_b6,post_str_b1,post_str_blast,is_rare_combination,reason,jumps_map,unijump
str,"array[u8, 6]",u8,u8,u8,u8,u8,u8,str,str,str,str,str,str,str,str,bool,u8,str,i64
"""152933404143""","[15, 29, … 43]",15,29,33,40,41,43,"""15""","""29""","""33""","""40""","""41""","""43""","""15""","""42""",false,0,"""14_04_07_01_02""",38
